<a href="https://colab.research.google.com/github/AliAkbarBadri/pokemon-gan/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [18]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation, Conv2DTranspose, Reshape, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image, ImageOps
import glob
import time
import matplotlib.gridspec as gridspec
from google.colab import files

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Data

In [7]:
! mkdir "./data"

In [8]:
from PIL import Image
import glob
data_dir = "/content/drive/My Drive/nn/gan/data"

for file in glob.glob(data_dir + '/*.png'):
    img = Image.open(file)
    jpg = img.convert('RGB')
    jpg.save('./data/' + file.split('/')[-1].split('.')[0] + '.jpg')

In [9]:
! cp -r "./data/" "/content/drive/My Drive/nn/gan/data_jpg"

In [42]:
def load_data(image_shape, batch_size):
    image_data_generator = ImageDataGenerator()
    dataset_path = "/content/drive/My Drive/nn/gan/data_jpg"
    dataset_generator = image_data_generator.flow_from_directory(
        dataset_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size,
        class_mode=None)
    return dataset_generator

In [ ]:
for file in glob.glob(data_dir + '/*.png'):
    img = Image.open(file)
    jpg = img.convert('RGB')
    jpg.save('./data/' + file.split('/')[-1].split('.')[0] + '.jpg')

In [21]:
! mkdir "./new_images"

mkdir: cannot create directory ‘./new_images’: File exists


In [22]:
def save_images(generated_images, epoch_no, batch_no):
    plt.figure(figsize=(8, 8), num=2)
    gs1 = gridspec.GridSpec(8, 8)
    gs1.update(wspace=0, hspace=0)

    for i in range(64):
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        image = generated_images[i, :, :, :]
        image += 1
        image *= 127.5
        fig = plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

    plt.tight_layout()
    save_name = 'new_images/generated_epoch' + str(
        epoch_no + 1) + '_batch' + str(batch_no + 1) + '.png'
    if not os.path.exists('new_images'):
        os.mkdir('new_images')
    plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
    plt.pause(0.0000000001)
    plt.show()

In [55]:
def get_generator(image_shape,codings_size=100):
    generator = Sequential([
      keras.layers.Dense(100, activation="selu", input_shape=[codings_size],name="gen_dense1"),
      keras.layers.Dense(150, activation="selu",name="gen_dense2"),
      keras.layers.Dense(image_shape[0] * image_shape[1]*image_shape[2], activation="sigmoid",name="gen_dense3"),
      keras.layers.Reshape(image_shape,name="gen_reshape")
    ],name="GEN_MODEL")
    # generator.compile(loss='binary_crossentropy',
    #                   optimizer="rmsprop",
    #                   metrics=None)
    # generator.
    return generator

def get_discriminator(image_shape):
    discriminator = keras.models.Sequential([
      keras.layers.Flatten(input_shape=image_shape,name="dis_flatten"),
      keras.layers.Dense(150, activation="selu",name="dis_dense1"),
      keras.layers.Dense(100, activation="selu",name="dis_dense2"),
      keras.layers.Dense(1, activation="sigmoid",name="dis_dense3")
    ],name = "DISC_MODEL")
    # discriminator.
    # optimizer = Adam(lr=0.0002, beta_1=0.5)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer="rmsprop",
                          metrics=None)
    return discriminator

def get_adversarial(generator, discriminator):
    gan = Sequential()
    discriminator.trainable = False
    gan.add(generator)
    gan.add(discriminator)

    # optimizer = Adam(lr=0.00015, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer="rmsprop",
                metrics=None)
    return gan

In [78]:
temp = np.random.normal(0, 1, size=(32,) + (1, 1, 100))
temp2 = np.random.normal(0, 1,
                                      size=(32 * 2,) +
                                          (1, 1, 100))
temp3 = (np.ones(32 * 2) -
                      np.random.random_sample(32 * 2) * 0.2)
temp3.shape

(64,)

# Train

In [79]:
def train(epochs, batch_size, image_shape):
    generator = get_generator(image_shape)
    generator.summary()
    discriminator = get_discriminator(image_shape)
    discriminator.summary()
    gan = get_adversarial(generator, discriminator)

    # Load dataset
    dataset_generator = load_data(image_shape, batch_size)

    number_of_batches = 11

    # Variables that will be used to plot the losses from the discriminator and
    # the adversarial models
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # Allow plot updates inside for loop
    plt.ion()

    current_batch = 0

    # Begin training
    for epoch in range(epochs):
        print("Epoch " + str(epoch + 1) + "/" + str(epochs) + " :")
        # g_batch_loss = []
        # d_batch_loss = []
        for batch_number in range(number_of_batches):

            start_time = time.time()

            real_images = dataset_generator.next()

            # Normalize the images between -1 and 1
            real_images /= 127.5
            real_images -= 1

            # The last batch is smaller than the other ones, so we need to
            # take that into account
            current_batch_size = real_images.shape[0]

            # Generate noise
            noise = np.random.normal(0, 1,
                                      size=(current_batch_size,) + (1, 1, 100)).reshape(current_batch_size,100)

            # Generate images
            # print(noise.shape)
            generated_images = generator(noise)

            # Add some noise to the labels that will be
            # fed to the discriminator
            real_y = (np.ones(current_batch_size) -
                      np.random.random_sample(current_batch_size) * 0.2)
            fake_y = np.random.random_sample(current_batch_size) * 0.2

            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            y2 = tf.constant([[1.]] * batch_size)

            # Let's train the discriminator
            discriminator.trainable = True

            d_loss = discriminator.train_on_batch(real_images, real_y)
            d_loss += discriminator.train_on_batch(generated_images, fake_y)

            discriminator_loss = np.append(discriminator_loss, d_loss)

            # Now it's time to train the generator
            discriminator.trainable = False

            noise = np.random.normal(0, 1,
                                      size=(current_batch_size * 2,) +
                                          (1, 1, 100)).reshape(current_batch_size * 2,100)

            # We try to mislead the discriminator by giving the opposite labels
            fake_y = (np.ones(current_batch_size * 2) -
                      np.random.random_sample(current_batch_size * 2) * 0.2)

            g_loss = gan.train_on_batch(noise, fake_y)
            adversarial_loss = np.append(adversarial_loss, g_loss)
            batches = np.append(batches, current_batch)

            # Each 50 batches show and save images
            if ((batch_number + 1) % 50 == 0 and
                    current_batch_size == batch_size):
                save_images(generated_images, epoch, batch_number)

            time_elapsed = time.time() - start_time

            if batch_number == number_of_batches-1:
              # print("aaaaaaaa",batch_number, number_of_batches)
              # Display and plot the results
              print("     Batch " + str(batch_number + 1) + "/" +
                    str(number_of_batches) +
                    " generator loss | discriminator loss : " +
                    str(g_loss) + " | " + str(d_loss) + ' - batch took ' +
                    str(time_elapsed) + ' s.')

            current_batch += 1

        # Save the model weights each 5 epochs
        # if (epoch + 1) % 5 == 0:
        #     discriminator.trainable = True
        #     if not os.path.exists('models'):
        #         os.mkdir('models')
        #     generator.save('models/generator_epoch' + str(epoch) + '.hdf5')
        #     discriminator.save('models/discriminator_epoch' +
        #                         str(epoch) + '.hdf5')
        
        # # Each epoch update the loss graphs
        # plt.figure(1)
        # plt.plot(batches, adversarial_loss, color='green',
        #           label='Generator Loss')
        # plt.plot(batches, discriminator_loss, color='blue',
        #           label='Discriminator Loss')
        # plt.title("GAN Training")
        # plt.xlabel("Batch Iteration")
        # plt.ylabel("Loss")
        # if epoch == 0:
        #     plt.legend()
        # plt.pause(0.0000000001)
        # plt.show()
        # plt.savefig('trainingLossPlot.png')
        

In [80]:
data_size = 352
batch_size = 64
image_shape = (64, 64,3)
coding_size = 100
epochs = 10
train(epochs, batch_size, image_shape)

Model: "GEN_MODEL"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gen_dense1 (Dense)           (None, 100)               10100     
_________________________________________________________________
gen_dense2 (Dense)           (None, 150)               15150     
_________________________________________________________________
gen_dense3 (Dense)           (None, 12288)             1855488   
_________________________________________________________________
gen_reshape (Reshape)        (None, 64, 64, 3)         0         
Total params: 1,880,738
Trainable params: 1,880,738
Non-trainable params: 0
_________________________________________________________________
Model: "DISC_MODEL"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dis_flatten (Flatten)        (None, 12288)             0         
___________________________

# HOML Model 1 (MLP, MLP)

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

codings_size = 30

generator = keras.models.Sequential([
    keras.layers.Dense(100, activation="selu", input_shape=[codings_size]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(28 * 28, activation="sigmoid"),
    keras.layers.Reshape([28, 28])
])
discriminator = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(150, activation="selu"),
    keras.layers.Dense(100, activation="selu"),
    keras.layers.Dense(1, activation="sigmoid")
])
gan = keras.models.Sequential([generator, discriminator])

In [ ]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
discriminator.trainable = False
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [ ]:
batch_size = 32
dataset = tf.data.Dataset.from_tensor_slices(X_train).shuffle(1000)
dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(1)

In [ ]:
def train_gan(gan, dataset, batch_size, codings_size, n_epochs=50):
    generator, discriminator = gan.layers
    for epoch in range(n_epochs):
        print("Epoch {}/{}".format(epoch + 1, n_epochs))              
        for X_batch in dataset:
            # phase1 - training the discriminator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            generated_images = generator(noise)
            X_fake_and_real = tf.concat([generated_images, X_batch], axis=0)
            y1 = tf.constant([[0.]] * batch_size + [[1.]] * batch_size)
            discriminator.trainable = True
            discriminator.train_on_batch(X_fake_and_real, y1)
            # phase2 - training the generator
            noise = tf.random.normal(shape=[batch_size, codings_size])
            y2 = tf.constant([[1.]] * batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y2)
        plot_multiple_images(generated_images, 8)                     
        plt.show()                                                    

In [ ]:
train_gan(gan, dataset, batch_size, codings_size, n_epochs=1)

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

noise = tf.random.normal(shape=[batch_size, codings_size])
generated_images = generator(noise)
plot_multiple_images(generated_images, 8)
save_fig("gan_generated_images_plot", tight_layout=False)

In [ ]:
train_gan(gan, dataset, batch_size, codings_size)

In [ ]:

import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os

np.random.seed(42)
tf.random.set_seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "gan"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")